<a href="https://colab.research.google.com/github/sspaulc/sturdy-computing-machine/blob/main/Breast_Cancer_Detection_with_97_accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential


In [ ]:
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization #batch normalization will standardize the data
from tensorflow.keras.layers import Conv1D, MaxPool1D # Conv1D summarizes data in 1D
 
from tensorflow.keras.optimizers import Adam #updates network weights based on training data. It is built on SGD but faster



In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import datasets,metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
cancer= datasets.load_breast_cancer()

In [ ]:
print(cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry 
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 3 is Mean Radius, f

In [ ]:
data_cancer= pd.DataFrame(data= cancer.data,columns= cancer.feature_names)
data_cancer.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
 targets=cancer.target 
 targets

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

In [ ]:
cancer.target_names

array(['malignant', 'benign'], dtype='<U9')

In [ ]:
#0= malignant, 1= benign

In [ ]:
data_cancer.shape

(569, 30)

In [ ]:
# 569 rows,30 columns array

In [ ]:
cancer_train,cancer_test, targets_train, targets_test = train_test_split(data_cancer,targets,test_size=0.2,random_state=0,stratify=y)
#training and testing in symmetric manner

In [ ]:
cancer_train.shape

(455, 30)

In [ ]:
cancer_test.shape

(114, 30)

In [ ]:
scaler= StandardScaler()
cancer_train=scaler.fit_transform(cancer_train) #fit_transform will fit data and then transform it. Fit data by calculating the mean
#and then replace the missing data 
cancer_test=scaler.transform(cancer_test)

In [ ]:
cancer_train=cancer_train.reshape(455,30,1)
cancer_test=cancer_test.reshape(114,30,1)

**Build** **the** **model**

In [ ]:
epochs=63 #63 times the algorithm will work through the entire dataset
model= Sequential() #linear stacks of layers with each layer having only one input and one output tensor
model.add(Conv1D(filters=32, kernel_size=2,activation= 'relu',input_shape=(30,1))) #takes 1D sequence of data 
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv1D(filters=64, kernel_size=2,activation= 'relu' )) #takes 1D sequence of data 
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten()) #Flatten to vector
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(1,activation='sigmoid'))


In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 29, 32)            96        
_________________________________________________________________
batch_normalization_7 (Batch (None, 29, 32)            128       
_________________________________________________________________
dropout_8 (Dropout)          (None, 29, 32)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 28, 64)            4160      
_________________________________________________________________
batch_normalization_8 (Batch (None, 28, 64)            256       
_________________________________________________________________
dropout_9 (Dropout)          (None, 28, 64)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 1792)             

In [ ]:
early_stopping=tf.keras.callbacks.EarlyStopping(patience=2)

In [ ]:
model.compile(optimizer= Adam(lr=0.00005),loss= 'binary_crossentropy',metrics=['accuracy'])
history=model.fit(cancer_train,targets_train,epochs=epochs,callbacks=[early_stopping],validation_data= (cancer_test,targets_test),verbose=1)

Epoch 1/63
15/15 [==============================] - 1s 20ms/step - loss: 0.0881 - accuracy: 0.9780 - val_loss: 0.0989 - val_accuracy: 0.9561
Epoch 2/63
15/15 [==============================] - 0s 8ms/step - loss: 0.0992 - accuracy: 0.9704 - val_loss: 0.0992 - val_accuracy: 0.9561
Epoch 3/63
15/15 [==============================] - 0s 10ms/step - loss: 0.0865 - accuracy: 0.9722 - val_loss: 0.0998 - val_accuracy: 0.9561
